In [ ]:
import requests
import json

page = requests.get("https://api.datamuse.com/words?rel_rhy=funny")
print(type(page))
print(page.text[:150]) # print the first 150 characters
print(page.url) # print the url that was fetched
print("------")
x = page.json() # turn page.text into a python object
print(type(x))
print("---first item in the list---")
print(x[0])
print("---the whole list, pretty printed---")
print(json.dumps(x, indent=2)) # pretty print the results

<class 'requests.models.Response'>
[{"word":"money","score":4415,"numSyllables":2},{"word":"honey","score":1206,"numSyllables":2},{"word":"sunny","score":717,"numSyllables":2},{"word":"
https://api.datamuse.com/words?rel_rhy=funny
------
<class 'list'>
---first item in the list---
{'word': 'money', 'score': 4415, 'numSyllables': 2}
---the whole list, pretty printed---
[
  {
    "word": "money",
    "score": 4415,
    "numSyllables": 2
  },
  {
    "word": "honey",
    "score": 1206,
    "numSyllables": 2
  },
  {
    "word": "sunny",
    "score": 717,
    "numSyllables": 2
  },
  {
    "word": "bunny",
    "score": 702,
    "numSyllables": 2
  },
  {
    "word": "blini",
    "score": 613,
    "numSyllables": 2
  },
  {
    "word": "gunny",
    "score": 449,
    "numSyllables": 2
  },
  {
    "word": "tunny",
    "score": 301,
    "numSyllables": 2
  },
  {
    "word": "sonny",
    "score": 286,
    "numSyllables": 2
  },
  {
    "word": "dunny",
    "score": 245,
    "numSyllables": 2


In order to not make mistakes in writing query parameters we can use dictionary.

In [ ]:
import requests

# page = requests.get("https://api.datamuse.com/words?rel_rhy=funny")
kval_pairs = {'rel_rhy': 'funny'}
page = requests.get("https://api.datamuse.com/words", params=kval_pairs)
print(page.text[:150]) # print the first 150 characters
print(page.url) # print the url that was fetched


There are a couple of other optional parameters for the function requests_with_caching.get().

    cache_file– it’s value should be a string specifying the name of

1.   the file containing the permanent cache. If you don’t specify anything, the default value is “permanent_cache.txt”. For the datamuse API, we’ve provide a cache in a file called datamuse_cache.txt. It just contains the saved response to the query for “https://api.datamuse.com/words?rel_rhy=funny”.
2.   private_keys_to_ignore– its value should be a list of strings. These are keys from the parameters dictionary that should be ignored when deciding whether the current request matches a previous request. The main purpose of this is that it allows us to return a result from the cache for some REST APIs that would otherwise require you to provide an API key in order to make a request. By default, it is set to [“api_key”], which is a query parameter used with the flickr API. You should not need to set this optional parameter.




    

This module is not available outside this textbook; in a full python environment you won’t be able to install a requests_with_caching module. But you can copy the code and make it work outside the textbook environment.

In [ ]:
import requests
import json

PERMANENT_CACHE_FNAME = "permanent_cache.txt"
TEMP_CACHE_FNAME = "this_page_cache.txt"

def _write_to_file(cache, fname):
    with open(fname, 'w') as outfile:
        outfile.write(json.dumps(cache, indent=2))

def _read_from_file(fname):
    try:
        with open(fname, 'r') as infile:
            res = infile.read()
            return json.loads(res)
    except:
        return {}

def add_to_cache(cache_file, cache_key, cache_value):
    temp_cache = _read_from_file(cache_file)
    temp_cache[cache_key] = cache_value
    _write_to_file(temp_cache, cache_file)

def clear_cache(cache_file=TEMP_CACHE_FNAME):
    _write_to_file({}, cache_file)

def make_cache_key(baseurl, params_d, private_keys=["api_key"]):
    """Makes a long string representing the query.
    Alphabetize the keys from the params dictionary so we get the same order each time.
    Omit keys with private info."""
    alphabetized_keys = sorted(params_d.keys())
    res = []
    for k in alphabetized_keys:
        if k not in private_keys:
            res.append("{}-{}".format(k, params_d[k]))
    return baseurl + "_".join(res)

def get(baseurl, params={}, private_keys_to_ignore=["api_key"], permanent_cache_file=PERMANENT_CACHE_FNAME, temp_cache_file=TEMP_CACHE_FNAME):
    full_url = requests.requestURL(baseurl, params)
    cache_key = make_cache_key(baseurl, params, private_keys_to_ignore)
    # Load the permanent and page-specific caches from files
    permanent_cache = _read_from_file(permanent_cache_file)
    temp_cache = _read_from_file(temp_cache_file)
    if cache_key in temp_cache:
        print("found in temp_cache")
        # make a Response object containing text from the change, and the full_url that would have been fetched
        return requests.Response(temp_cache[cache_key], full_url)
    elif cache_key in permanent_cache:
        print("found in permanent_cache")
        # make a Response object containing text from the change, and the full_url that would have been fetched
        return requests.Response(permanent_cache[cache_key], full_url)
    else:
        print("new; adding to cache")
        # actually request it
        resp = requests.get(baseurl, params)
        # save it
        add_to_cache(temp_cache_file, cache_key, resp.text)
        return resp

In [ ]:
import requests_with_caching
# it's not found in the permanent cache
res = requests_with_caching.get("https://api.datamuse.com/words?rel_rhy=happy", permanent_cache_file="datamuse_cache.txt")
print(res.text[:100])
# this time it will be found in the temporary cache
res = requests_with_caching.get("https://api.datamuse.com/words?rel_rhy=happy", permanent_cache_file="datamuse_cache.txt")
# This one is in the permanent cache.
res = requests_with_caching.get("https://api.datamuse.com/words?rel_rhy=funny", permanent_cache_file="datamuse_cache.txt")


ModuleNotFoundError: ignored

['money', 'honey', 'sunny']


https://www.flickr.com/services/api/
for searching for pics
documentation: https://www.flickr.com/services/api/flickr.photos.search.html

In [ ]:
# import statements
import requests
# import webbrowser

# apply for a flickr authentication key at http://www.flickr.com/services/apps/create/apply/?
# paste the key (not the secret) as the value of the variable flickr_key
flickr_key = '0efb8ce35a3c9836f60c088536ff5038'

def get_flickr_data(tags_string):
    baseurl = "https://api.flickr.com/services/rest/"
    params_diction = {}
    params_diction["api_key"] = flickr_key # from the above global variable
    params_diction["tags"] = tags_string # must be a comma separated string to work correctly
    params_diction["tag_mode"] = "all"
    params_diction["method"] = "flickr.photos.search"
    params_diction["per_page"] = 5
    params_diction["media"] = "photos"
    params_diction["format"] = "json"
    params_diction["nojsoncallback"] = 1
    flickr_resp = requests.get(baseurl, params = params_diction)
    # Useful for debugging: print the url! Uncomment the below line to do so.
    print(flickr_resp.url) # Paste the result into the browser to check it out...
    return flickr_resp.json()

result_river_mts = get_flickr_data("river,mountains")
print(result_river_mts)
# Some code to open up a few photos that are tagged with the mountains and river tags...

photos = result_river_mts['photos']['photo']
for photo in photos:
    owner = photo['owner']
    photo_id = photo['id']
    url = 'https://www.flickr.com/photos/{}/{}'.format(owner, photo_id)
    print(url)
    # webbrowser.open(url)


https://api.flickr.com/services/rest/?api_key=0efb8ce35a3c9836f60c088536ff5038&tags=river%2Cmountains&tag_mode=all&method=flickr.photos.search&per_page=5&media=photos&format=json&nojsoncallback=1
{'photos': {'page': 1, 'pages': 43230, 'perpage': 5, 'total': 216148, 'photo': [{'id': '52894495733', 'owner': '195047443@N07', 'secret': 'a442df6b96', 'server': '65535', 'farm': 66, 'title': 'Landmannalaugar', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '52892892259', 'owner': '90318937@N05', 'secret': '6a2cdb91ef', 'server': '65535', 'farm': 66, 'title': 'little white house', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '52892774034', 'owner': '29304855@N05', 'secret': '5f58be294d', 'server': '65535', 'farm': 66, 'title': 'Twelth Night - Coniston Water', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0}, {'id': '52892995490', 'owner': '29304855@N05', 'secret': '4611d3bd5e', 'server': '65535', 'farm': 66, 'title': 'Out on the water - Coniston', 'ispublic': 1, 'isfriend': 0, 'isfam

In [ ]:
#import statements
import requests
# import webbrowser

# apply for a flickr authentication key at http://www.flickr.com/services/apps/create/apply/?
# paste the key (not the secret) as the value of the variable flickr_key
flickr_key = '0efb8ce35a3c9836f60c088536ff5038'

def get_flickr_data(tags_string):
    baseurl = "https://api.flickr.com/services/rest/"
    params_diction = {}
    params_diction["api_key"] = flickr_key # from the above global variable
    params_diction["tags"] = tags_string # must be a comma separated string to work correctly
    params_diction["tag_mode"] = "all"
    params_diction["method"] = "flickr.photos.search"
    params_diction["per_page"] = 2
    params_diction["media"] = "photos"
    params_diction["format"] = "json"
    params_diction["nojsoncallback"] = 1
    flickr_resp = requests.get(baseurl, params = params_diction)
    # Useful for debugging: print the url! Uncomment the below line to do so.
    print(flickr_resp.url) # Paste the result into the browser to check it out...
    return flickr_resp.json()

result_river_mts = get_flickr_data("sun,mountains")
print(json.dumps(result_river_mts))
# Some code to open up a few photos that are tagged with the mountains and river tags...
photos = result_river_mts['photos']['photo']
for photo in photos:
    owner = photo['owner']
    photo_id = photo['id']
    url = 'https://www.flickr.com/photos/{}/{}'.format(owner, photo_id)
    print(url)
    # webbrowser.open(url)


https://api.flickr.com/services/rest/?api_key=0efb8ce35a3c9836f60c088536ff5038&tags=sun%2Cmountains&tag_mode=all&method=flickr.photos.search&per_page=2&media=photos&format=json&nojsoncallback=1
{"photos": {"page": 1, "pages": 77481, "perpage": 2, "total": 154962, "photo": [{"id": "52893849141", "owner": "142609644@N04", "secret": "fe4dab5f5e", "server": "65535", "farm": 66, "title": "albanian mountain village", "ispublic": 1, "isfriend": 0, "isfamily": 0}, {"id": "52890753822", "owner": "196647052@N08", "secret": "53fd59f8e5", "server": "65535", "farm": 66, "title": "Het indrukwekkende schouwspel van de ondergaande zon achter de bergen. The impressive spectacle of the sun setting behind the mountains. Le spectacle impressionnant du coucher de soleil derri\u00e8re les montagnes .", "ispublic": 1, "isfriend": 0, "isfamily": 0}]}, "stat": "ok"}
https://www.flickr.com/photos/142609644@N04/52893849141
https://www.flickr.com/photos/196647052@N08/52890753822


In [ ]:
import requests

base_url = "http://api.openweathermap.org/geo/1.0/direct"
param_dic = {}

param_dic["q"]= "london"
param_dic["limit"]="6"

param_dic["appid"]="2e7bb4eb08c7c47641449fc69d2b03b0"
openweathermap_res=requests.get(base_url,params=param_dic)
print(openweathermap_res.url)

http://api.openweathermap.org/geo/1.0/direct?q=london&limit=6&appid=2e7bb4eb08c7c47641449fc69d2b03b0


In [ ]:
import requests_with_caching
import json

parameters = {"term": "Ann Arbor", "entity": "podcast"}
iTunes_response = requests_with_caching.get("https://itunes.apple.com/search", params = parameters, permanent_cache_file="itunes_cache.txt")

py_data = json.loads(iTunes_response.text)
for r in py_data['results']:
    print(r['trackName'])
